## 6.10 Homework

The goal of this homework is to create a tree-based regression model for prediction apartment prices (column `'price'`).

In this homework we'll again use the New York City Airbnb Open Data dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

Let's load the data:

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]
!wget "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv"
df = pd.read_csv('AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

--2021-10-17 22:55:16--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘raw.githubusercontent.com’


* Apply the log tranform to `price`
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1

In [5]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
df.price = np.log1p(df['price'])
train_val, test = train_test_split(df, test_size=0.2, random_state = 1)
train, val = train_test_split(train_val, test_size = 0.25, random_state =1)

train_y = train['price']
del train['price']
val_y = val['price']
del val['price']
test_y = test['price']
del test['price']

Now, use `DictVectorizer` to turn train and validation into matrices:

In [6]:
dv = DictVectorizer(sparse=False)
train_dict = train.to_dict(orient='records')
val_dict = val.to_dict(orient='records')
test_dict = test.to_dict(orient='records')
train_X = dv.fit_transform(train_dict)
val_X = dv.transform(val_dict)
test_X = dv.transform(test_dict)

## Question 1

Let's train a decision tree regressor to predict the price variable. 

* Train a model with `max_depth=1`

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
dt = DecisionTreeRegressor(max_depth=1)

In [8]:
dt.fit(train_X, train_y.values)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=1,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [9]:
print(dt.feature_importances_)
print(dv.feature_names_)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
['availability_365', 'calculated_host_listings_count', 'latitude', 'longitude', 'minimum_nights', 'neighbourhood_group=Bronx', 'neighbourhood_group=Brooklyn', 'neighbourhood_group=Manhattan', 'neighbourhood_group=Queens', 'neighbourhood_group=Staten Island', 'number_of_reviews', 'reviews_per_month', 'room_type=Entire home/apt', 'room_type=Private room', 'room_type=Shared room']


Which feature is used for splitting the data?

* `room_type`
* `neighbourhood_group`
* `number_of_reviews`
* `reviews_per_month`

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1`  (optional - to make training faster)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(train_X, train_y)

val_pred = rf.predict(val_X)

mse = mean_squared_error(val_y, val_pred)
rmse = mse ** (0.5)
print(mse, rmse)

0.21239922245530993 0.46086790130720745


What's the RMSE of this model on validation?

* 0.059
* 0.259
* 0.459
* 0.659

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10
* Set `random_state` to `1`
* Evaluate the model on the validation dataset

In [11]:
n_estimators = np.arange(10, 201, 10)
print(n_estimators)

rmses = []
for n_estimator in n_estimators:
    rf = RandomForestRegressor(n_estimators=n_estimator, random_state=1, n_jobs=-1)
    rf.fit(train_X, train_y)
    
    val_pred = rf.predict(val_X)
    mse = mean_squared_error(val_y, val_pred)
    rmse = mse ** (0.5)
    rmses.append((n_estimator, rmse))

[ 10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180
 190 200]


In [12]:
print(np.round(rmses, 3))

[[ 10.      0.461]
 [ 20.      0.447]
 [ 30.      0.444]
 [ 40.      0.443]
 [ 50.      0.442]
 [ 60.      0.441]
 [ 70.      0.441]
 [ 80.      0.441]
 [ 90.      0.44 ]
 [100.      0.44 ]
 [110.      0.439]
 [120.      0.439]
 [130.      0.439]
 [140.      0.439]
 [150.      0.439]
 [160.      0.439]
 [170.      0.439]
 [180.      0.439]
 [190.      0.439]
 [200.      0.439]]


After which value of `n_estimators` does RMSE stop improving?

- 10
- 50
- 70
- 120

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`

In [13]:
max_depths = np.array([10, 15 ,20, 25])
n_estimators = np.arange(10, 201, 10)
rmses = []
for max_depth in max_depths:
    for n_estimator in n_estimators:
        rf = RandomForestRegressor(n_estimators=n_estimator, max_depth=max_depth, random_state=1, n_jobs=-1)
        rf.fit(train_X, train_y)
    
        val_pred = rf.predict(val_X)
        mse = mean_squared_error(val_y, val_pred)
        rmse = mse ** (0.5)
        rmses.append((max_depth, n_estimator, rmse))
        


In [14]:
smallest = None
for rmse in rmses:
    if smallest == None:
        smallest = rmse
    if rmse[2] < smallest[2]:
        smallest = rmse
    print(rmse)

print(smallest)

(10, 10, 0.4452143747042935)
(10, 20, 0.44186025446736055)
(10, 30, 0.44119702899693847)
(10, 40, 0.4412961755096089)
(10, 50, 0.44094334533104307)
(10, 60, 0.4410092231841634)
(10, 70, 0.440805067072908)
(10, 80, 0.4406687544140446)
(10, 90, 0.44038357073730117)
(10, 100, 0.4401528976805655)
(10, 110, 0.44009660818755353)
(10, 120, 0.43987740067533315)
(10, 130, 0.4399162343095045)
(10, 140, 0.4399110603581997)
(10, 150, 0.43973464764653036)
(10, 160, 0.43968677200268014)
(10, 170, 0.4396630337408857)
(10, 180, 0.43978564033104756)
(10, 190, 0.43972758422457403)
(10, 200, 0.4397243291806476)
(15, 10, 0.45031956641701915)
(15, 20, 0.4411594782927354)
(15, 30, 0.43957294137292663)
(15, 40, 0.43878049796761576)
(15, 50, 0.4380556774400964)
(15, 60, 0.4377240141366398)
(15, 70, 0.43732890888456977)
(15, 80, 0.437187281858576)
(15, 90, 0.43672813376233555)
(15, 100, 0.4364144325084769)
(15, 110, 0.4361411926004894)
(15, 120, 0.43613457181897897)
(15, 130, 0.43619977849712344)
(15, 140, 0.4

What's the best `max_depth`:

* 10
* 15
* 20
* 25

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [15]:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(train_X, train_y)
features = list(zip(rf.feature_importances_, dv.feature_names_))
print(sorted(features, key=lambda x : x[0]))

[(0.0001265046108617081, 'neighbourhood_group=Staten Island'), (0.00024589339434527964, 'neighbourhood_group=Bronx'), (0.0011030640502402431, 'neighbourhood_group=Brooklyn'), (0.001162209878223539, 'neighbourhood_group=Queens'), (0.00448830752851972, 'room_type=Shared room'), (0.004576840746812983, 'room_type=Private room'), (0.030950509087841915, 'calculated_host_listings_count'), (0.03413554841532696, 'neighbourhood_group=Manhattan'), (0.04304398048096192, 'number_of_reviews'), (0.052659710954611125, 'reviews_per_month'), (0.053666576448405945, 'minimum_nights'), (0.07633571528032984, 'availability_365'), (0.15201924532350514, 'latitude'), (0.15358906161021973, 'longitude'), (0.391896832189794, 'room_type=Entire home/apt')]


What's the most important feature? 

* `neighbourhood_group=Manhattan`
* `room_type=Entire home/apt`	
* `longitude`
* `latitude`

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

In [20]:
import xgboost as xgb
features = dv.get_feature_names()
dtrain = xgb.DMatrix(train_X, label=train_y, feature_names=features)
dval = xgb.DMatrix(val_X, label=val_y, feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1
}
model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)

[0]	train-rmse:3.02752	val-rmse:3.02415
[1]	train-rmse:2.14667	val-rmse:2.14390
[2]	train-rmse:1.53878	val-rmse:1.53721
[3]	train-rmse:1.12557	val-rmse:1.12523
[4]	train-rmse:0.85100	val-rmse:0.85174
[5]	train-rmse:0.67490	val-rmse:0.67752
[6]	train-rmse:0.56687	val-rmse:0.57148
[7]	train-rmse:0.50448	val-rmse:0.51139
[8]	train-rmse:0.46913	val-rmse:0.47777
[9]	train-rmse:0.45009	val-rmse:0.45965
[10]	train-rmse:0.43912	val-rmse:0.44981
[11]	train-rmse:0.43327	val-rmse:0.44475
[12]	train-rmse:0.42936	val-rmse:0.44210
[13]	train-rmse:0.42668	val-rmse:0.44038
[14]	train-rmse:0.42463	val-rmse:0.43943
[15]	train-rmse:0.42259	val-rmse:0.43827
[16]	train-rmse:0.42113	val-rmse:0.43772
[17]	train-rmse:0.42074	val-rmse:0.43787
[18]	train-rmse:0.41896	val-rmse:0.43744
[19]	train-rmse:0.41812	val-rmse:0.43726
[20]	train-rmse:0.41716	val-rmse:0.43691
[21]	train-rmse:0.41499	val-rmse:0.43645
[22]	train-rmse:0.41437	val-rmse:0.43611
[23]	train-rmse:0.41403	val-rmse:0.43614
[24]	train-rmse:0.41391	va

Now change `eta` first to `0.1` and then to `0.01`

In [21]:
xgb_params['eta'] = 0.1
model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)

[0]	train-rmse:3.87217	val-rmse:3.86889
[1]	train-rmse:3.49150	val-rmse:3.48840
[2]	train-rmse:3.14949	val-rmse:3.14635
[3]	train-rmse:2.84232	val-rmse:2.83951
[4]	train-rmse:2.56650	val-rmse:2.56412
[5]	train-rmse:2.31905	val-rmse:2.31692
[6]	train-rmse:2.09714	val-rmse:2.09526
[7]	train-rmse:1.89834	val-rmse:1.89663
[8]	train-rmse:1.72033	val-rmse:1.71878
[9]	train-rmse:1.56120	val-rmse:1.55976
[10]	train-rmse:1.41910	val-rmse:1.41786
[11]	train-rmse:1.29248	val-rmse:1.29149
[12]	train-rmse:1.17977	val-rmse:1.17907
[13]	train-rmse:1.07974	val-rmse:1.07936
[14]	train-rmse:0.99113	val-rmse:0.99118
[15]	train-rmse:0.91299	val-rmse:0.91348
[16]	train-rmse:0.84421	val-rmse:0.84524
[17]	train-rmse:0.78390	val-rmse:0.78525
[18]	train-rmse:0.73111	val-rmse:0.73308
[19]	train-rmse:0.68507	val-rmse:0.68776
[20]	train-rmse:0.64528	val-rmse:0.64883
[21]	train-rmse:0.61109	val-rmse:0.61518
[22]	train-rmse:0.58175	val-rmse:0.58648
[23]	train-rmse:0.55655	val-rmse:0.56186
[24]	train-rmse:0.53529	va

In [22]:
xgb_params['eta'] = 0.01
model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)

[0]	train-rmse:4.25336	val-rmse:4.25010
[1]	train-rmse:4.21141	val-rmse:4.20815
[2]	train-rmse:4.16988	val-rmse:4.16661
[3]	train-rmse:4.12877	val-rmse:4.12551
[4]	train-rmse:4.08807	val-rmse:4.08481
[5]	train-rmse:4.04779	val-rmse:4.04454
[6]	train-rmse:4.00792	val-rmse:4.00467
[7]	train-rmse:3.96845	val-rmse:3.96521
[8]	train-rmse:3.92937	val-rmse:3.92615
[9]	train-rmse:3.89070	val-rmse:3.88749
[10]	train-rmse:3.85242	val-rmse:3.84921
[11]	train-rmse:3.81452	val-rmse:3.81133
[12]	train-rmse:3.77701	val-rmse:3.77382
[13]	train-rmse:3.73988	val-rmse:3.73671
[14]	train-rmse:3.70313	val-rmse:3.69996
[15]	train-rmse:3.66674	val-rmse:3.66359
[16]	train-rmse:3.63073	val-rmse:3.62759
[17]	train-rmse:3.59508	val-rmse:3.59195
[18]	train-rmse:3.55979	val-rmse:3.55666
[19]	train-rmse:3.52487	val-rmse:3.52175
[20]	train-rmse:3.49030	val-rmse:3.48719
[21]	train-rmse:3.45608	val-rmse:3.45298
[22]	train-rmse:3.42220	val-rmse:3.41910
[23]	train-rmse:3.38867	val-rmse:3.38559
[24]	train-rmse:3.35548	va

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* 0.01

## Submit the results


Submit your results here: https://forms.gle/wQgFkYE6CtdDed4w8

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline


The deadline for submitting is 20 October 2021, 17:00 CET (Wednesday). After that, the form will be closed.

